In [2]:
import random

REWARD_CLICK = 1
REWARD_NO_CLICK = 0


class WebsiteEnvironmentSimple:
    def __init__(self, proba):
        self.proba = proba

    def do(self, action):
        reward = REWARD_CLICK if random.random() < self.proba[action] else REWARD_NO_CLICK
        return reward

In [3]:
from abc import ABC, abstractmethod


class BanditBase(ABC):
    def __init__(self, K, env):
        self.arms = list(range(K))
        self.env = env
        self.history = []

    @abstractmethod
    def get_action(self):
        raise NotImplementedError()

    def update(self, arm, reward):
        self.history.append([arm, reward])
        self._update(arm, reward)

    @abstractmethod
    def _update(self, arm, reward):
        raise NotImplementedError()


In [4]:
import random
import numpy as np
import math
from abc import abstractmethod

class UGapEBandit(BanditBase):
    def __init__(self, ϵ, m, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ϵ = ϵ
        self.m = m
        
        self.rewards = {arm: [] for arm in self.arms}
    
    def get_regret_bound(self):
        betas_arms = []
        bounds_arms = []
        for arm in self.arms:
            arm_beta = self.get_arm_beta(arm)
            betas_arms.append(arm_beta)
            bounds_arms.append((
                np.mean(self.rewards[arm]) - arm_beta,
                np.mean(self.rewards[arm]) + arm_beta
            ))
        
        regret_bound = []
        for arm_k in set(self.arms):
            values = []
            for arm_i in set(self.arms) - {arm}:
                U_i = bounds_arms[arm_i][1]
                L_k = bounds_arms[arm_k][0]
                values.append(U_i - L_k)
            regret_bound.append(max(values))
                                
        return regret_bound, bounds_arms, betas_arms
        
    
    def get_action(self):
        # Makes sure we have played every arm at least once
        for arm, rewards in self.rewards.items():
            if len(rewards) == 0:
                return arm
            
        B_t, bounds_arms, β_t = self.get_regret_bound()
        L_t = [b[0] for b in bounds_arms]
        U_t = [b[1] for b in bounds_arms]
        J_t = np.argsort(B_t)[:self.m]

        u_t = list(set(self.arms) - set(J_t))[np.argmax([
            U_t[arm] for arm in set(self.arms) - set(J_t)
        ])]
        l_t = J_t[np.argmax([
            L_t[arm] for arm in J_t
        ])]
        
        arm = u_t if β_t[u_t] > β_t[l_t] else l_t

        return arm

    def _update(self, arm, reward):
        self.rewards[arm].append(reward)

class UGapEBudgetBandit(UGapEBandit):
    def __init__(self, ϵ, m, n, a, *args, **kwargs):
        super().__init__(ϵ=ϵ, m=m, *args, **kwargs)
        self.n = n
        self.a = a
    
    def get_arm_beta(self, arm):
        # b = 1
        return math.sqrt(self.a / len(self.rewards[arm]))

In [14]:
conversion_rates = [0.15, 0.01, 0.02, 0.03]
env = WebsiteEnvironmentSimple(conversion_rates)

In [15]:
bandit = UGapEBudgetBandit(ϵ=0.3, m=1, K=4, n=4, a=1/2, env=env)

In [16]:
data = []
for _ in range(2000):
    action = bandit.get_action()
    reward = env.do(action)
    data.append({
        'action': action,
        'reward': reward
    })
    bandit.update(action, reward)

In [17]:
import pandas as pd
data = pd.DataFrame(data)
data.groupby('action').count()

,reward
action,
0,774
1,150
2,302
3,774


Kinda weird \
Is this what it means to do best arm identification instead of regret minimization? If so, do website designers really want this?